In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import sys
import os

In [ ]:
###############################################################################
# Housekeeping
###############################################################################
dir_input = os.path.join("./inputs")
dir_metsim = os.path.join("./processing")
dir_out = dir_metsim

if not os.path.exists(dir_out):
  os.makedirs(dir_out)

fname_obs = "RelativeHumidityDataforBC.xlsx"
fname_model = "Timeseries_step1_drybulb,raw_cloud,raw_relhum.csv"

dt0_obs = datetime(1973, 1, 1, 0, 0)
dt1_obs = datetime(2015, 12, 31, 23, 0)

dt0_model = datetime(1915, 1, 1, 0, 0)
dt1_model = datetime(2015, 12, 31, 23, 0)

mode_list = ["Model", "Obs"]

# Print to csv
icsv = 1

# For plot
dpif = 200
num_bins = 200 #CDF bins

# Outlier to be removed
outlier_cut = 100
print("Outlier_cut: " + str(outlier_cut))

bias_correction = "Quantile_Mapping"

num_percentile = 50

## Read input files

In [ ]:
df_obs = pd.read_excel(os.path.join(dir_input, fname_obs))
df_model = pd.read_csv(os.path.join(dir_metsim, fname_model),
                       usecols=["Date", "Relative Humidity [%]"])

In [ ]:
##########################################################################
# Create index in correct format using pandas date_range
##########################################################################

# Observed
dt0 = df_obs["Date"].iloc[0]
dt1 = df_obs["Date"].iloc[-1]
index = pd.date_range(dt0, dt1, freq = timedelta(hours = 1))
df_obs = df_obs.set_index(index)
df_obs = df_obs.drop("Date", axis = 1)

# Model
dt0 = df_model["Date"].iloc[0]
dt1 = df_model["Date"].iloc[-1]
index = pd.date_range(dt0, dt1, freq = timedelta(hours = 1))
df_model = df_model.set_index(index)
df_model = df_model.drop("Date", axis = 1)

In [ ]:
df_initial = pd.concat([df_model, df_obs], axis=1)

## Clean up the input file

In [ ]:
df_raw = df_initial.copy()

In [ ]:
##########################################################################
# Replace column names for convenience
##########################################################################
print(df_raw.columns)
df_raw = df_raw.rename(columns = {df_raw.columns[0]: mode_list[0],
                                  df_raw.columns[1]: mode_list[1]})

# Generate Timeseries

In [ ]:
dict_ts = {}
for mode in mode_list:

  dict_ts[mode] = {}
  
  print("Generating timeseries: " + mode)

  ##########################################################################
  # Hourly data
  ##########################################################################

  #
  # Remove outliers by replacing them with NaN
  #

  # Check max value
  df_nonan = df_raw[~pd.isnull(df_raw[mode])]
  print("Max: " + str(max(df_nonan[mode])))

  # Identify outliers
  df_remove = df_raw[df_raw >= outlier_cut]

  # Extract non-NaN values
  df_remove = df_remove[~pd.isnull(df_remove[mode])]

  # Extract unique values
  list_remove = df_remove[mode].tolist()
  set_remove = set(list_remove)
  list_remove = list(set_remove)

  # Replace them to NaN
  if (len(list_remove) > 0):
    print("Outliers:", list_remove)
    df_raw = df_raw.replace(list_remove, np.NaN)
  else:
    pass

  df_hourly = df_raw[mode]

  ############################################################################
  # Save timeseries into dictionary for later use
  ############################################################################
  dict_ts[mode] = {"Hourly": df_hourly}

# Bias Correction

## Quantile Mapping

In [ ]:
col0 = "Model (Adj)"

dict_ts_adj = {}

print("Generating timeseries for adjusted model output")

dict_ts_adj = {}
############################################################################
# Hourly timeseries
############################################################################
df_tmp = dict_ts["Obs"]["Hourly"][dt0_obs:dt1_obs]
df_tmp = df_tmp.replace(0, np.nan)
df_obs_hourly = df_tmp.dropna()

df_model_hourly = dict_ts["Model"]["Hourly"][dt0_model:dt1_model].dropna()

arr_obs = df_obs_hourly.to_numpy()
arr_model = df_model_hourly.to_numpy()
df_map = df_model_hourly

list_model = []

prange = int(100/num_percentile)
for upb in range(prange, 100+prange, prange):

  pct_obs = np.percentile(arr_obs, upb)
  pct_model = np.percentile(arr_model, upb)
  lowb = upb - prange

  pct_model_low = np.percentile(arr_model, lowb)

  dat_model = df_map[df_map.between(pct_model_low,
                  pct_model,
                  inclusive="both")]
  
  ratio = pct_obs/pct_model
  if ( np.isnan(ratio) or np.isinf(ratio)):
    continue
  else:
    dat_model_new = dat_model * ratio
    list_model.append(dat_model_new)

df_tmp = pd.concat(list_model, axis = 0)
df_tmp = df_tmp.sort_index()

# check for duplicate
dup_index = df_tmp[df_tmp.index.duplicated(keep=False)]
if (len(dup_index) > 0):
  print("Duplicate index found:", dup_index)
  print("Dropping all but first")

  df_tmp = df_tmp[~df_tmp.index.duplicated(keep='first')]

# Check
arr_model = df_tmp.to_numpy()
for upb in range(prange, 100+prange, prange):

  pct_obs = np.percentile(arr_obs, upb)
  pct_model = np.percentile(arr_model, upb)

df_hourly = df_tmp.to_frame()
df_hourly = df_hourly.rename(columns = {df_hourly.columns[0]: col0})

dict_ts_adj["Hourly"] = df_hourly[col0]

## Remove outliers

In [ ]:
dict_ts_adj["Hourly_Filtered"] = pd.DataFrame(data = None)
print(len(dict_ts_adj["Hourly_Filtered"]))
df_hourly = dict_ts_adj["Hourly"]

##############################################################################
# Remove outliers
##############################################################################
# Check max value
df_nonan = df_hourly[~pd.isnull(df_hourly)]

# Identify outliers
df_remove = df_hourly[df_hourly > outlier_cut]

# Extract non-NaN values
df_remove = df_remove[~pd.isnull(df_remove)]

# Extract unique values
list_remove = df_remove.tolist()
set_remove = set(list_remove)
list_remove = list(set_remove)

# Replace them to NaN
if (len(list_remove) > 0):
  print("Outliers:", list_remove)
  df_hourly = df_hourly.replace(list_remove, np.NaN)

  # Check max value
  df_nonan = df_hourly[~pd.isnull(df_hourly)]
  print("Maximum after removing outliers: " + str(max(df_nonan)))

  dict_ts_adj["Hourly_Filtered"] = df_hourly

else:
  pass

print(len(dict_ts_adj["Hourly_Filtered"]))

## Print adjusted values to csv

In [ ]:
if (icsv == 1):
  if (len(dict_ts_adj["Hourly_Filtered"]) > 0):
    df_print = pd.DataFrame(dict_ts_adj["Hourly"])
    df_print.index.name = "Date"    
    dict_ts_adj["Hourly"].to_csv(os.path.join(dir_out,
                    "Timeseries_step2_relhum_with_outliers.csv"),
                    float_format = "%.2f", na_rep = "NaN",
                    header=["Relative Humidity [%]"])

    df_print = pd.DataFrame(dict_ts_adj["Hourly_Filtered"])
    df_print.index.name = "Date"        
    dict_ts_adj["Hourly_Filtered"].to_csv(os.path.join(dir_out,
                    "Timeseries_step2_relhum_without_outliers.csv"),
                    float_format = "%.2f", na_rep = "NaN",
                    header=["Relative Humidity [%]"])
  else:
    df_print = pd.DataFrame(dict_ts_adj["Hourly"])
    df_print.index.name = "Date"
    df_print.to_csv(os.path.join(dir_out,
                    "Timeseries_step2_corrected_relhum.csv"),
                    float_format = "%.2f", na_rep = "NaN",
                    header=["Relative Humidity [%]"])
      

# Plot

## Cumulative

In [ ]:
if (len(dict_ts_adj["Hourly_Filtered"]) > 0):
  keys = ["Hourly", "Hourly_Filtered"]
else:
  keys = ["Hourly"]

for key in keys:
  data_obs = dict_ts["Obs"]["Hourly"][dt0_obs:dt1_obs]
  data_model = dict_ts["Model"]["Hourly"][dt0_model:dt1_model]
  
  data_adj = dict_ts_adj[key][dt0_model:dt1_model]

  plt.figure(figsize = (6, 6), dpi = 200)

  data = data_obs.dropna()
  values, base = np.histogram(data, bins=num_bins)
  cumulative = np.cumsum(values)/len(data)
  plt.plot(base[:-1], cumulative, "C0")

  data = data_model.dropna()
  values, base = np.histogram(data, bins=num_bins)
  cumulative = np.cumsum(values)/len(data)
  plt.plot(base[:-1], cumulative, "C1")

  data = data_adj.dropna()
  values, base = np.histogram(data, bins=num_bins)
  cumulative = np.cumsum(values)/len(data)
  plt.plot(base[:-1], cumulative, "C2")

  plt.title("Relative Humidity")
  plt.xlabel("Relative Humidity [%]")
  plt.legend(["Observation", "Model (Raw)", "Model (Adj.)"])
  plt.ylim([0, 1])
  plt.xlim([0, 100])
  plt.grid()